In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
#import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
import functions
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
best_positions = np.genfromtxt("best_pos.csv", delimiter="; ")
print(type(best_positions[0]))
print(best_positions)

<class 'numpy.ndarray'>
[[-3.97290234  5.19235956  0.5       ]
 [-8.11327289  5.93962456  0.5       ]
 [-8.51439548  6.62725701  0.5       ]
 [-8.37032305  6.50910438  0.5       ]
 [-6.91224171  5.80460312  0.5       ]
 [-8.62191844  6.96890901  0.5       ]
 [-5.65194545 -2.03943957  0.5       ]
 [-6.64277137  5.35201411  0.5       ]
 [-6.12203445  5.93117367  0.5       ]]


In [3]:
pcd_flat = o3d.io.read_point_cloud("pcd_flat.pcd")
valid_area = o3d.io.read_point_cloud("valid_area.pcd")
valid_area_vg = o3d.geometry.VoxelGrid.create_from_point_cloud(valid_area, 0.2)
o3d.visualization.draw_geometries([pcd_flat, valid_area_vg])

In [7]:
markers = []

for position in best_positions:
    position[2] = 0
    print(position)
    markers.append(functions.create_box_at_point(position, size=(0.5,0.5,0.5)))
    
o3d.visualization.draw_geometries([pcd_flat, valid_area_vg]+markers)

[-3.97290234  5.19235956  0.        ]
[-8.11327289  5.93962456  0.        ]
[-8.51439548  6.62725701  0.        ]
[-8.37032305  6.50910438  0.        ]
[-6.91224171  5.80460312  0.        ]
[-8.62191844  6.96890901  0.        ]
[-5.65194545 -2.03943957  0.        ]
[-6.64277137  5.35201411  0.        ]
[-6.12203445  5.93117367  0.        ]


In [5]:
inclusion = []
for marker in markers:
    bb = marker.get_oriented_bounding_box()
    points = bb.get_box_points()
    print(np.asarray(points))
    inc = valid_area_vg.check_if_included(points)
    inc_sum = sum(inc)
    if inc_sum >= 4:
        inclusion.append(True)
    else:
        inclusion.append(False)
    print(inc_sum)

[[-3.82290234  5.64235956 -0.25      ]
 [-3.52290234  5.04235956 -0.25      ]
 [-3.82290234  5.64235956  0.25      ]
 [-4.42290234  5.34235956 -0.25      ]
 [-4.12290234  4.74235956  0.25      ]
 [-4.42290234  5.34235956  0.25      ]
 [-4.12290234  4.74235956 -0.25      ]
 [-3.52290234  5.04235956  0.25      ]]
0
[[-8.29002651  6.36628072  0.26057275]
 [-8.54281325  5.75618667  0.25104003]
 [-8.29002651  6.37421335 -0.2471158 ]
 [-7.68373254  6.11512982  0.25664852]
 [-7.93651928  5.5129684  -0.26057275]
 [-7.68373254  6.12306245 -0.25104003]
 [-7.93651928  5.50503577  0.2471158 ]
 [-8.54281325  5.7641193  -0.25664852]]
0
[[-8.51439548  6.87725701  0.5       ]
 [-8.01439548  6.87725701  0.        ]
 [-8.51439548  6.37725701  0.5       ]
 [-9.01439548  6.87725701  0.        ]
 [-8.51439548  6.37725701 -0.5       ]
 [-9.01439548  6.37725701  0.        ]
 [-8.51439548  6.87725701 -0.5       ]
 [-8.01439548  6.37725701  0.        ]]
3
[[-8.22032305  6.95910438 -0.25      ]
 [-7.92032305  6

In [6]:
for i in range(len(best_positions)):
    best_positions[i][2] = 0
pos_utility_vec = o3d.utility.Vector3dVector(best_positions)

#inclusion = valid_area_vg.check_if_included(pos_utility_vec)
print(inclusion)
for i in range(len(markers)):
    if not inclusion[i]:
        markers[i].paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([pcd_flat, valid_area_vg]+markers)

[False, False, False, False, False, False, False, False, False]
